# File
## I. file object
#### I.1 应用场景
- python的built-in function<code>open()</code>的返回值是一个file类型的object。
#### I.2 file object特征
1. file object本质上是一个link，连接到机器上的指定文件。
2. file object既是一个iterable，又是一个iterator，遍历的单位是文件中的line。
   - 因为它的implement中既有<code>\_\_iter__ </code>, 又有<code>\_\_next__ </code> method
   - 和其他built-in iterable不同的是，每次open function得到的file object只能遍历一次，如果要再次遍历，就需要重新open或者用<code>file.seek(0) </code>回到文件的起始位置。
3. file和string类型联系紧密。
   - python用file object提供的method读写文件。不管是用<code>f.read()</code>读取文件内容，还是<code>f.write()</code>将内容写入file，这些'内容'在python中都是以string的形式处理的，可能是str string，也可能是byte string。

## II. 用法
### II.1 open file
- <code>open('file_name', 'mode_arg', encoding='encode_arg')</code>: 返回file object
- 参数说明：
  - <code>file_name</code>：通常用raw string来表示文件所在位置的path，比如：<code>r'~/Document/test.txt'</code>
  - <code>mode_arg</code>：
    - 默认是<code>r</code>，表示只读
    - <code>w</code>，表示只写，如果原文本中有内容的话，会被cover掉
    - <code>w+</code>，表示读写
    - <code>a</code>，表示在原文本的最后继续写入新的内容
    - <code>rb</code>, <code>wb</code>，分别表示以raw byte方式读/写文件中的内容
  - <code>encode_arg</code>：指定读取文件的编码方式<font color=blue>[说明详见string部分的notes]</font>

- open file最好用Context Management<code> with</code>。with statement对应的block结束后会自动close file，即使有error occurs during file operations也会执行close操作。这样就不用手动call f.close()，减少bug。
  - <code>with open(...) as f:
             f.read(...)</code>

In [15]:
# 注意，源文件顶部标记的编码方式不是指源文件本身在存为file时的编码方式
# 下面例子中'latin-1'和'utf-16'是不同的概念，'utf-16'是源文件本身存为file时使用的编码
# 'latin-1'该源文件运行时，文件中涉及的string类型object的赋值内容，下例中的'x'和'y'使用的编码方式
s = """
# -*- coding: latin-1 -*-

x = chr(0xc4) + chr(0xe8)  # x和y是相同的两个拉丁字符
y = '\xc4\xe8'             # '\xc4\xe8'是这两个字符对应的latin-1码值
for i in x, y:
    print(f"{i},{i.encode()},  {i.encode('latin-1')}")
""" 
# encoding参数指定了写入string的编码方式
with open('string_sample.txt', 'w', encoding='utf-16') as f: # 等价于f = open(...)
    f.write(s)

In [13]:
# 读取文件中string
with open('string_sample.txt', 'r', encoding='utf-16') as f:  # 必须指定utf-16
    content = f.read()    
print(content)


# -*- coding: latin-1 -*-

x = chr(0xc4) + chr(0xe8)  # x和y是相同的两个拉丁字符
y = 'Äè'             # 'Äè'是这两个字符对应的latin-1码值
for i in x, y:
    print(f"{i},{i.encode()},  {i.encode('latin-1')}")



In [14]:
# open in  to see raw bytes
with open('string_sample.txt', 'rb') as f:  # 这个raw bytes不是unicode的raw bytes，
    content = f.read()                      # 是文本本身编码形式的raw bytes
print(content)

b'\xff\xfe\n\x00#\x00 \x00-\x00*\x00-\x00 \x00c\x00o\x00d\x00i\x00n\x00g\x00:\x00 \x00l\x00a\x00t\x00i\x00n\x00-\x001\x00 \x00-\x00*\x00-\x00\n\x00\n\x00x\x00 \x00=\x00 \x00c\x00h\x00r\x00(\x000\x00x\x00c\x004\x00)\x00 \x00+\x00 \x00c\x00h\x00r\x00(\x000\x00x\x00e\x008\x00)\x00 \x00 \x00#\x00 \x00x\x00\x8cTy\x00/f\xf8v\x0cT\x84v$N*N\xc9b\x01NW[&{\n\x00y\x00 \x00=\x00 \x00\'\x00\xc4\x00\xe8\x00\'\x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00#\x00 \x00\'\x00\xc4\x00\xe8\x00\'\x00/f\xd9\x8f$N*NW[&{\xf9[\x94^\x84vl\x00a\x00t\x00i\x00n\x00-\x001\x00\x01x<P\n\x00f\x00o\x00r\x00 \x00i\x00 \x00i\x00n\x00 \x00x\x00,\x00 \x00y\x00:\x00\n\x00 \x00 \x00 \x00 \x00p\x00r\x00i\x00n\x00t\x00(\x00f\x00"\x00{\x00i\x00}\x00,\x00{\x00i\x00.\x00e\x00n\x00c\x00o\x00d\x00e\x00(\x00)\x00}\x00,\x00 \x00 \x00{\x00i\x00.\x00e\x00n\x00c\x00o\x00d\x00e\x00(\x00\'\x00l\x00a\x00t\x00i\x00n\x00-\x001\x00\'\x00)\x00}\x00"\x00)\x00\n\x00'


### II.2 file object的常用 method
- 最主要的功能是实现文件的读写。
#### 1. read
1. <code>s = f.read()</code>: 读整个文档，输出为1个string 
2. <code>s = f.read(N)</code>: 最多读N个character，输出为1个string
3. <code>f.readline()</code>：读1行，包括行末的'\n'
4. <code>f.readlines()</code>：读整个文档，输出list of line strings，每行都有行末的'\n'
- <font color=red>尽量不要用<code>read()或者readline()</code>，因为他们都会一次读整个文档到memory，存储效率很低。</font><font color=green>**用后面的file iterator。**</font>

#### 2. write
1. <code>f.write(s)</code>：将string内容写入file
2. <code>f.writelines(list_of_strings)</code>：将参数list of strings中的所有line string都写入文件

#### 3. flush
- flush output buffer to disk without closing
- files are buffered.意思是说，在默认情况下，write语句写入file的内容不会在执行语句时马上全部写完，只有close file或者手动执行flush method才会立即一次写完。

#### 4. file iterator
- 尽管有read method，但<font color=blue>按行读取file的最好方式通常是直接用file iterator</font>。
  - 典型方式是：
    - for loop: <code>for line in f: use line</code>
    - list comprehension: <code>func(line) for line in f</code>

### II.3 读写content的类型转换
#### II.3.1 类型转换需求
- <font color=blue>读写的具体内容在python中都是用string类型来表示。</font>
  - <code>f.read()</code>返回的是string。<code>f.write(s)</code>是将string写入指定文件
  - 由于read得到的是string对象，很可能不是合适的处理对象，就需要将string转换成int或者float等合适的处理类型。
    - 比如csv文件中本来是digits，可以用<code> float(s)</code>函数来转变类型。
    - image文件要处理的是0-255的digits，可以用<code>int(s)</code>来转换
    - 用<code>eval('[a, 2, 3]')</code>可以将<code>'[a, 2, 3]'</code>转变成list
  - write文件的时候也要提前将要写入的内容转换成string之后再写入。
    - 可以用<code>str()</code>将原内容转换成string
- <font color=blue>python提供了标准库来处理这类问题:</font>
    - **pickle module**处理generic object storage
    - **struct module**处理packed binary data

#### II.3.2 pickle module处理python object
- 功能：执行object serialization，方便地实现将python object写入文件或者将文件中的内容读入并转化成python object。<font color=red>不用在读后和写前手动做python object到string之间的转换。</font>
- <font color=blue>pickle在处理python object转换的时候，不是用的str string，而是用的byte string。所以用pickle的时候，open file时的参数一定要指定为binary格式。</font>
- 风险：pickle可以直接把binary byte读成python object，这意味着，如果file中有恶意代码，pickle将他们读成python代码段后可能产生安全风险。
- 应用限制：

```python
# 用pickle将object写入文件
import pickle
f = open('file_name', 'wb') # pickle需要指定为读或写binary的格式
pickle.dump(obj_name, f)
```

```python
# 等价于执行了：
s = str(obj_name)
f = open('file_name', 'w')  # 这里不是binary data，而是utf-8，所以写入的内容格式和pickle不同
f.write(s_obj)
```

```python
# 用pickle从文件读取object，这是体现pickle优势的地方
f = open('file_name', 'rb') # pickle需要指定为读或写binary的格式
scr = pickle.load(f)
```

```python
# 等价于执行了：
f = open('file_name', 'r')  # 这里读进来的文件需要是utf-8编码格式的文件
s = read(f)
# 之后要用string method + eval()来把string恢复成python object，这里会很复杂
```


#### II.3.3 json module可以处理JSON format
- pickle可以处理任意python object，但它只能用于python，不能处理其他语言。pickle把python中定义的object转换成一种专有的format，再处理成string。从string转换成object的时候也一样以这个python自己定义的专有format为中介。<font color=red>这就限制了write出来的file的使用范围。</font>
- 另一种可以跨语言使用的数据转换格式是JSON。python的json module可以将**部分python object**转变成JSON格式。虽然不支持有所python object，但优点是跨语言可用。很多语料库都用的JSON格式。
  - 最初是为JavaScript设计的。JSON表示JavaScriptObjectNotation。
- **json支持的python对象类型**：
  - collections都支持: string, dictionary, list, tuple. <font color=red>不支持set</font>
  - numbers有: int, float, boolean, None

#### II.3.4 struct module处理packed binary data
- 如果file中的内容是binary data，比如网络传输数据，那么可以用struct module来compose和parse这些数据。
- 略